In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle

In [2]:
def getMap(maps={}, needs='a', maxA=3, cash=()):
    if len(cash) == 0:
        n = maxA - 1
        s = [(0, 0) for _ in range(maxA)]
    else:
        n = cash[0]
        s = cash[1]

    if n == 0:
        for i in range(5):
            for j in range(5):
                s[n] = (i, j)
                if needs == 'a':
                    maps[str(s)] = [0] * maxA
                else:
                    maps[str(s)] = 0
    else:
        for i in range(5):
            for j in range(5):
                s[n] = (i, j)
                getMap(maps, needs, maxA, (n - 1, s.copy()))
    return maps

In [3]:
class getActionandProb:
    def __init__(self, action, pe):
        self.action = action
        self.length = len(action)
        self.curr = [0] * self.length
        self.pes = [pe ** i * (0.25 - pe / 4) ** (self.length - i) for i in range(self.length + 1)]
        self.ite = 0

    def __iter__(self):
        return self

    def __next__(self):
        x = self.curr.copy()
        c = sum([i == j for i, j in zip(x, self.action)])
        if self.ite < 5 ** self.length:
            self.ite += 1
            for i in range(self.length - 1, -1, -1):
                if self.curr[i] + 1 == 5:
                    self.curr[i] = 0
                else:
                    self.curr[i] += 1
                    break
            return x, self.pes[c]
        else:
            raise StopIteration

In [32]:
class func_approx():
    def __init__(self, my_map, n_agents):
        self.n = len(my_map)
        self.m = len(my_map[0])
        self.my_map = my_map
        self.n_agents = n_agents

        # get target position, change to a unique code
        for i in range(self.n):
            for j in range(self.m):
                if type(my_map[i][j]) == type('s'):
                    if my_map[i][j] == 'Rs':
                        self.Rs = i * 1000 + j
                    if my_map[i][j] == 'Rd':
                        self.Rd = i * 1000 + j

        # initialize
        self.theta = np.zeros([4, 1])
        self.theta_old = np.random.rand(*[4, 1])

    def getFeatures(self, state):
        features = np.zeros_like(self.theta)
        states = eval(state)
        states_code = []

        # change state to unique code and if in negtice state and if in target
        for s in states:
            tmp_code = s[0] * 1000 + s[1]
            states_code.append(tmp_code)
            # negative state
            if type(self.my_map[s[0]][s[1]]) == type('s') and self.my_map[s[0]][s[1]] == 'Rw':
                features[1] += 1
            # wall
            if type(self.my_map[s[0]][s[1]]) == type(0) and self.my_map[s[0]][s[1]] == -1:
                features[2] += 1
            # in target
            if tmp_code in [self.Rs, self.Rd]:
                features[0] += 1

        # if collision
        features[3] = len(states_code) - len(set(states_code))

        return features

    def getValues(self, state):
        features = self.getFeatures(state)
        return (self.theta.T.dot(features))[0, 0]

    def updateTheta(self, Data, Y):
        # here Data is N*n_features
        #    Y      is N*1
        self.theta_old = self.theta.copy()
        self.theta = (np.linalg.inv(Data.T.dot(Data) + 0.0001 * np.identity(Data.shape[1]))).dot(Data.T).dot(Y)

In [33]:
moves_map = {0: [0, 0], 1: [-1, 0], 2: [0, 1], 3: [1, 0], 4: [0, -1]}
def getValues(state, acts, my_map, n, m, faFandler):
    next_state = []
    for s, a in zip(eval(state), acts):
        # if in wall
        if type(my_map[s[0]][s[1]]) == type(0) and my_map[s[0]][s[1]] == -1:
            next_state.append(s)
        else:
            # next pos
            movement = moves_map[a]
            next_x = s[0] + movement[0]
            next_y = s[1] + movement[1]

            # if not valid: out the map
            if (next_x < 0 or next_x >= n) or (next_y < 0 or next_y >= m):
                next_x = s[0]
                next_y = s[1]
            # else if target is obstacle
            if type(my_map[next_x][next_y]) == type(0) and my_map[next_x][next_y] == -1:
                next_x = s[0]
                next_y = s[1]

            # next value
            next_state.append([next_x, next_y])

    return faFandler.getValues(str(next_state))

In [34]:
def VI(my_map, pe, discount, horizon, rewards_map, n_agents=2, p1=0.1, p2=0.1):
    # prepare the vlaues
    n = len(my_map)
    m = len(my_map[0])

    # the fa
    faHandler = func_approx(my_map, n_agents)

    # states
    states = getMap(maps={}, needs='v', maxA=n_agents)
    X = np.zeros([len(states), faHandler.theta.shape[0]])
    Y = np.zeros([len(states), 1])

    # actions
    actions = []
    for a, _ in getActionandProb([0] * n_agents, pe):
        actions.append(a)
    tmp_action_value = [0] * len(actions)

    # ite = 0
    ite = 0
    while (np.linalg.norm(faHandler.theta_old - faHandler.theta) >= 1e-4 and ite < horizon) or ite == 0:
        choose = np.random.rand(len(states)) <= p1
        for num, state in enumerate(states.keys()):
            # for selected state:
            if choose[num]:
                # for each action, get its next value
                for i, action in enumerate(actions):
                    tmp_val = 0
                    tmp_p = 0

                    # get next value
                    for acts, p in getActionandProb(action, pe):
                        if np.random.rand() >= p2:
                            pass
                        else:
                            tmp_val += discount * p * getValues(state, acts, my_map, n, m, faHandler)
                            tmp_p += p
                    tmp_val /= tmp_p

                    # assign
                    tmp_action_value[i] = tmp_val

                # get final assign
                X[num,] = faHandler.getFeatures(state)[:,0]
                Y[num, 0] = max(tmp_action_value)

                # get current reward
                for s in eval(state):
                    Y[num, 0] += rewards_map[my_map[s[0]][s[1]]]

                # get collision
                Y[num, 0] += X[num, 3] * rewards_map['Rc']
            else:
                pass
        # update
        faHandler.updateTheta(X[choose,], Y[choose,])
        ite += 1
        print('iteration {}, result: '.format(ite))
        print(str(faHandler.theta))
    return faHandler

In [35]:
moves_map = {0: [0, 0], 1: [-1, 0], 2: [0, 1], 3: [1, 0], 4: [0, -1]}
my_map = [[0]*5 for _ in range(5)]
my_map[2][0] = 'Rs'
my_map[2][2] = 'Rd'
my_map[1][1] = -1
my_map[2][1] = -1
my_map[1][3] = -1
my_map[2][3] = -1
my_map[4][0] = 'Rw'
my_map[4][1] = 'Rw'
my_map[4][2] = 'Rw'
my_map[4][3] = 'Rw'
my_map[4][4] = 'Rw'

pe = 1.0
discount = 0.9
horizon = np.inf
rewards_map = {'Rd' : 10, 'Rs': 10, 'Rw': -1, 0:0, -1:-10, 'Rc':-5}

n_agents = 2
p1 = 1.0
p2 = 1.0

VI(my_map, pe, discount, horizon, rewards_map, n_agents, p1, p2)

iteration 1, result: 
[[ 9.99998938]
 [-1.00000022]
 [-9.99999562]
 [-4.99997961]]
iteration 2, result: 
[[ 1.95112697e+01]
 [ 1.52965095e-02]
 [-1.81286897e+01]
 [-4.86671759e+00]]
iteration 3, result: 
[[ 28.97143441]
 [  0.95069267]
 [-24.55408413]
 [ -2.69544259]]
iteration 4, result: 
[[ 38.63851749]
 [  2.71099528]
 [-29.34032402]
 [  0.04099088]]
iteration 5, result: 
[[ 48.61212533]
 [  5.28094346]
 [-32.57171847]
 [  3.3728703 ]]
iteration 6, result: 
[[ 58.86889629]
 [  9.1623038 ]
 [-34.43933371]
 [  8.63379725]]
iteration 7, result: 
[[ 69.59156493]
 [ 14.60165552]
 [-35.00738838]
 [ 15.78772814]]
iteration 8, result: 
[[ 80.97643539]
 [ 21.85818859]
 [-34.29924088]
 [ 24.87708957]]
iteration 9, result: 
[[ 93.23954903]
 [ 31.20432429]
 [-32.29960068]
 [ 36.01900654]]
iteration 10, result: 
[[106.62119078]
 [ 42.93628584]
 [-28.95729133]
 [ 49.40022763]]
iteration 11, result: 
[[121.39019875]
 [ 57.38860279]
 [-24.18711805]
 [ 65.27390379]]
iteration 12, result: 
[[137.8436

KeyboardInterrupt: 